In [ ]:
import os, sys
import numpy as np
import matplotlib.pyplot as mpl
import scipy as sp
import pandas as pd
from mayavi import mlab

In [ ]:
sys.path.append("./bin/")
import ZernikeFunc as ZF
import SurfaceFunc as SF
import MyUtils as MU  # my personal collection of functions

# Initial parameters

In [ ]:
# Parameters definition
Npixel = 25    # the plane edge in pixels
Rs = 6         # the radius of the sphere that includes the patch
Dpp = 0.5      # the distance between points of the same patch (needed to remove islands)
center = 5000  # point of the surface used as center of the patch (default index value)
ZOrder = 20    # the Zernike expansion order (needed only for Zernike Reconstruction)
threshold = 5  # value to enstablish variance is high (in Angstrom units)
Daa = 3        # threshold to find points in contact between two proteins (in Angstrom units)

# Contact area between two surfaces

In [ ]:
# To load surface and inizialize the surface class for protein A:
surf_name_a = "./data/3B0F_A_min.dms"
surf_a_ = pd.read_csv(surf_name_a)
l_a = len(surf_a_["x"])
print("Npoints Protein A =", l_a)
surf_a = np.zeros((l_a, 6))
surf_a[:,:] = surf_a_[["x", "y", "z", "Nx", "Ny", "Nz"]]
surf_a_obj = SF.Surface(surf_a[:,:], patch_num = 0, r0 = Rs, theta_max = 45)

# To load surface and inizialize the surface class for protein B:
surf_name_b = "./data/3B0F_B_min.dms"
surf_b_ = pd.read_csv(surf_name_b) 
l_b = len(surf_b_["x"])
print("Npoints Protein B =", l_b)
surf_b = np.zeros((l_b, 6))
surf_b[:,:] = surf_b_[["x", "y", "z", "Nx", "Ny", "Nz"]]
surf_b_obj = SF.Surface(surf_b[:,:], patch_num = 0, r0 = Rs, theta_max = 45)

In [ ]:
# Research of contact points between the two proteins
# in particular the patch center nearest to the center of mass of contact zone
center_a, patch_prot_a, center_b, patch_prot_b = MU.GroupNearPoints(Daa, surf_a_obj, surf_b_obj)

3D figures

In [ ]:
res1, c = SF.ConcatenateFigPlots([surf_a_obj.surface[:,:3],surf_b_obj.surface[:,:3]])
SF.Plot3DPoints(res1[:,0], res1[:,1], res1[:,2], c, 0.3)

In [ ]:
cm = patch_prot_a[center_a]
res1, c = SF.ConcatenateFigPlots([patch_prot_a,np.row_stack([cm,cm])])
SF.Plot3DPoints(res1[:,0], res1[:,1], res1[:,2], c, 0.3)

In [ ]:
cm = patch_prot_b[center_b]
res1, c = SF.ConcatenateFigPlots([patch_prot_b,np.row_stack([cm,cm])])
SF.Plot3DPoints(res1[:,0], res1[:,1], res1[:,2], c, 0.3)

To find the true indices of the points closest to the center of mass 

In [ ]:
# The indices of the points found by MU.GroupNearPoints are NOT referred to the entire surface of the relative protein, but are referred to the area of the contact surface.
# I use MU.PointNearPoint to identify indexes with respect to whole surface.
center_a_true = MU.PointNearPoint(surf_a[:,:3], patch_prot_a[center_a])
center_b_true = MU.PointNearPoint(surf_b[:,:3], patch_prot_b[center_b])
center_a = center_a_true
center_b = center_b_true
# center_a e center_b saranno usati anche in seguito

In [ ]:
# generation of patches with two methods
plane_W_a, plane_P_a, plane_W_b, plane_P_b = MU.PatchesMethods(Npixel, surf_a_obj, center_a, surf_b_obj, center_b, Dpp)

In [ ]:
# plots of patches with two methods
MU.PlotPatchesComparison("A", Npixel, Rs, plane_W_a, plane_P_a, center_a, Dpp, Daa, "", "")
MU.PlotPatchesComparison("B", Npixel, Rs, plane_W_b, plane_P_b, center_b, Dpp, Daa, "", "")

In [ ]:
# generation of processed patches
plane_W_a_proc = surf_a_obj.EnlargePixels( surf_a_obj.FillTheGap_everywhere(plane_=plane_W_a) )
plane_P_a_proc = surf_a_obj.EnlargePixels( surf_a_obj.FillTheGap_everywhere(plane_=plane_P_a) )
plane_W_b_proc = surf_a_obj.EnlargePixels( surf_a_obj.FillTheGap_everywhere(plane_=plane_W_b) )
plane_P_b_proc = surf_a_obj.EnlargePixels( surf_a_obj.FillTheGap_everywhere(plane_=plane_P_b) )

In [ ]:
# plots of processed patches
MU.PlotPatchesComparison("A", Npixel, Rs, plane_W_a_proc, plane_P_a_proc, center_a, Dpp, Daa, "", "")
MU.PlotPatchesComparison("B", Npixel, Rs, plane_W_b_proc, plane_P_b_proc, center_b, Dpp, Daa, "", "")

# Complementarity

In [ ]:
%%time

center_1 = center_a
center_2 = center_b

plane_W_1, plane_P_1, plane_W_2, plane_P_2 = MU.PatchesMethods(Npixel, surf_a_obj, center_1, surf_b_obj, center_2, Dpp)

coeff_diff_a = MU.ZernikeCoeff_Distance(ZOrder, surf_a_obj, plane_W_1, surf_a_obj, plane_P_1)
coeff_diff_b = MU.ZernikeCoeff_Distance(ZOrder, surf_b_obj, plane_W_2, surf_b_obj, plane_P_2)

_, perc_W_a = MU.PercHigherVariance_Weights("var", Npixel, surf_a_obj, center_1, Dpp, threshold)
_, perc_P_a = MU.PercHigherVariance_Projections("var", Npixel, surf_a_obj, center_1, Dpp, threshold)
_, perc_W_b = MU.PercHigherVariance_Weights("var", Npixel, surf_b_obj, center_2, Dpp, threshold)
_, perc_P_b = MU.PercHigherVariance_Projections("var", Npixel, surf_b_obj, center_2, Dpp, threshold)
perc_a = np.absolute( perc_W_a - perc_P_a )
perc_b = np.absolute( perc_W_b - perc_P_b )

print("Protein A: Patch center = {}".format(center_1))
print("Protein B: Patch center = {}".format(center_2))
print("Patch A: Difference Zernike coefficients = {},  perc_a = {}".format(coeff_diff_a,perc_a))
print("Patch B: Difference Zernike coefficients = {},  perc_b = {}".format(coeff_diff_b,perc_b))

In [ ]:
%%time

with open("inv_vs_perc.txt", "w") as last_file :
    for i in range(100) :
        center_1 = np.random.randint(5000,7000)
        center_2 = np.random.randint(5000,7000)
        plane_W_a, plane_P_a, plane_W_b, plane_P_b = MU.PatchesMethods(Npixel, surf_a_obj, center_1, surf_b_obj, center_2, Dpp)
        coeff_diff_a = MU.ZernikeCoeff_Distance(ZOrder, surf_a_obj, plane_W_a, surf_a_obj, plane_P_a)
        coeff_diff_b = MU.ZernikeCoeff_Distance(ZOrder, surf_b_obj, plane_W_b, surf_b_obj, plane_P_b)
        _, perc_W_a = MU.PercHigherVariance_Weights("var", Npixel, surf_a_obj, center_1, Dpp, threshold)
        _, perc_P_a = MU.PercHigherVariance_Projections("var", Npixel, surf_a_obj, center_1, Dpp, threshold)
        _, perc_W_b = MU.PercHigherVariance_Weights("var", Npixel, surf_b_obj, center_2, Dpp, threshold)
        _, perc_P_b = MU.PercHigherVariance_Projections("var", Npixel, surf_b_obj, center_2, Dpp, threshold)
        perc_a = np.absolute( perc_W_a - perc_P_a )
        perc_b = np.absolute( perc_W_b - perc_P_b )
        last_file.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(center_1, center_2, coeff_diff_a, coeff_diff_b, perc_a, perc_b))

In [ ]:
coeff_a = np.loadtxt("./risultati/inv_vs_perc.txt", usecols=2, unpack=True)
coeff_b = np.loadtxt("./risultati/inv_vs_perc.txt", usecols=3, unpack=True)
perc_a = np.loadtxt("./risultati/inv_vs_perc.txt", usecols=4, unpack=True)
perc_b = np.loadtxt("./risultati/inv_vs_perc.txt", usecols=5, unpack=True)

fig, ax = mpl.subplots(nrows=1, ncols=1, figsize=(8,4), facecolor="white", dpi=200)
ax.set_xlabel("Difference of percentage", fontsize="8")
ax.set_ylabel("Difference of invariants", fontsize="8")
ax.tick_params(axis="both", width ="0.60", color="black", labelsize="6")
ax.locator_params(axis="x", nbins=21)
ax.locator_params(axis="y", nbins=21)
for side in ax.spines.keys():  # 'top', 'bottom', 'left', 'right'
    ax.spines[side].set_linewidth(0.60)
    ax.spines[side].set_color("black")
ax.plot(perc_a, coeff_a, "o", markersize="1", label="Zone A", color="blue", rasterized=True)
ax.plot(perc_b, coeff_b, "o", markersize="1", label="Zone B", color="red", rasterized=True)
ax.legend(fontsize="7")
fig.tight_layout()
#mpl.savefig("inv_vs_perc.pdf")